IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M5\FAULT_M5_5


In [3]:
guasto=1

VOLO m5 FAULT 5%

In [4]:
rcou_m5_fault5 = pd.read_csv("RCOU.csv")
rcou_m5_fault5 = rcou_m5_fault5.astype("float64")
rcou_m5_fault5 = rcou_m5_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m5_fault5 = rcou_m5_fault5[((rcou_m5_fault5['C9'] >= potenza) & (rcou_m5_fault5['C10'] >= potenza) & (rcou_m5_fault5['C11'] >= potenza) & (rcou_m5_fault5['C12'] >= potenza) & (rcou_m5_fault5['C13'] >= potenza) & (rcou_m5_fault5['C14']>= potenza))]
rcou_m5_fault5=rcou_m5_fault5.reset_index(drop=True)
display(rcou_m5_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76590156.0,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0
1,76628157.0,1359.0,1426.0,1417.0,1370.0,1383.0,1404.0
2,76649076.0,1399.0,1414.0,1450.0,1360.0,1442.0,1369.0
3,76680796.0,1469.0,1379.0,1365.0,1480.0,1413.0,1438.0
4,76712365.0,1370.0,1510.0,1507.0,1375.0,1440.0,1449.0
...,...,...,...,...,...,...,...
4510,129967010.0,1590.0,1438.0,1512.0,1525.0,1563.0,1471.0
4511,129977954.0,1583.0,1423.0,1507.0,1510.0,1561.0,1451.0
4512,129991804.0,1538.0,1435.0,1457.0,1520.0,1486.0,1492.0
4513,130003524.0,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0


In [5]:
min=rcou_m5_fault5['TimeUS'][0]
max=rcou_m5_fault5['TimeUS'][len(rcou_m5_fault5)-1]
print(max)
print(min)

130015745.0
76590156.0


In [6]:
xkf1_m5_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m5_fault5 = xkf1_m5_fault5.astype("float64")
xkf1_m5_fault5 = xkf1_m5_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_fault5 = xkf1_m5_fault5[((xkf1_m5_fault5['TimeUS'] >= min) & (xkf1_m5_fault5['TimeUS'] <= max	))]
xkf1_m5_fault5 = xkf1_m5_fault5.reset_index(drop=True)
print(xkf1_m5_fault5)

           TimeUS  Roll  Pitch    Yaw
0      76598120.0  0.62  -0.26  17.98
1      76615343.0  0.65  -0.25  17.99
2      76625563.0  0.56  -0.26  17.98
3      76636156.0  0.52  -0.31  17.98
4      76647059.0  0.53  -0.33  17.97
...           ...   ...    ...    ...
4855  129964991.0 -0.62  -2.45   1.17
4856  129975220.0 -0.57  -2.47   1.15
4857  129989898.0 -0.34  -2.45   1.13
4858  130001095.0 -0.07  -2.44   1.10
4859  130013613.0  0.06  -2.35   1.07

[4860 rows x 4 columns]


In [7]:
att_m5_fault5 = pd.read_csv("ATT.csv")
att_m5_fault5 = att_m5_fault5.astype("float64")
att_m5_fault5 = att_m5_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_fault5 = att_m5_fault5[((att_m5_fault5['TimeUS'] >= min) & (att_m5_fault5['TimeUS'] <= max	))]
att_m5_fault5=att_m5_fault5.reset_index(drop=True)
print(att_m5_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      76600096.0     0.07  0.62     -0.03  -0.26   17.98  17.98
1      76616574.0     0.07  0.65     -0.04  -0.25   17.98  17.99
2      76627514.0     0.08  0.56     -0.04  -0.26   17.98  17.98
3      76638014.0     0.09  0.52     -0.04  -0.31   17.98  17.98
4      76648250.0     0.09  0.53     -0.04  -0.33   17.98  17.97
...           ...      ...   ...       ...    ...     ...    ...
4855  129966271.0     0.56 -0.62     -0.37  -2.45    5.87   1.17
4856  129977309.0     0.56 -0.57     -0.32  -2.47    5.87   1.15
4857  129991025.0     0.56 -0.34     -0.28  -2.45    5.87   1.13
4858  130002833.0     0.54 -0.07     -0.25  -2.44    5.87   1.10
4859  130014932.0     0.51  0.06     -0.23  -2.35    5.87   1.07

[4860 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_fault5)>len(xkf1_m5_fault5):
    to_add=att_m5_fault5[len(xkf1_m5_fault5):]
    att_m5_fault5=att_m5_fault5[:len(xkf1_m5_fault5)]

if len(xkf1_m5_fault5)>len(att_m5_fault5):
    to_add=xkf1_m5_fault5[len(att_m5_fault5):]

for idx,i in enumerate(att_m5_fault5['Roll']):
    if(xkf1_m5_fault5['Roll'][idx] != i):
        att_m5_fault5['Roll'][idx] = ((att_m5_fault5['Roll'][idx] + xkf1_m5_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m5_fault5['Pitch']):
    if(xkf1_m5_fault5['Pitch'][idx] != i):
        att_m5_fault5['Pitch'][idx] = ((att_m5_fault5['Pitch'][idx] + xkf1_m5_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_fault5['Yaw']):
    if(xkf1_m5_fault5['Yaw'][idx] != i):
        att_m5_fault5['Yaw'][idx] = ((att_m5_fault5['Yaw'][idx] + xkf1_m5_fault5['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_fault5=pd.concat([att_m5_fault5,to_add])

print(att_m5_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      76600096.0     0.07  0.62     -0.03  -0.26   17.98  17.98
1      76616574.0     0.07  0.65     -0.04  -0.25   17.98  17.99
2      76627514.0     0.08  0.56     -0.04  -0.26   17.98  17.98
3      76638014.0     0.09  0.52     -0.04  -0.31   17.98  17.98
4      76648250.0     0.09  0.53     -0.04  -0.33   17.98  17.97
...           ...      ...   ...       ...    ...     ...    ...
4855  129966271.0     0.56 -0.62     -0.37  -2.45    5.87   1.17
4856  129977309.0     0.56 -0.57     -0.32  -2.47    5.87   1.15
4857  129991025.0     0.56 -0.34     -0.28  -2.45    5.87   1.13
4858  130002833.0     0.54 -0.07     -0.25  -2.44    5.87   1.10
4859  130014932.0     0.51  0.06     -0.23  -2.35    5.87   1.07

[4860 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m5_fault5 = esc_0_m5_fault5.astype("float64")
esc_0_m5_fault5 = esc_0_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_fault5 = esc_0_m5_fault5[((esc_0_m5_fault5['TimeUS'] >= min) & (esc_0_m5_fault5['TimeUS'] <= max))]
esc_0_m5_fault5=esc_0_m5_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m5_fault5 = esc_1_m5_fault5.astype("float64")
esc_1_m5_fault5 = esc_1_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_fault5 = esc_1_m5_fault5[((esc_1_m5_fault5['TimeUS'] >= min) & (esc_1_m5_fault5['TimeUS'] <= max))]
esc_1_m5_fault5=esc_1_m5_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m5_fault5 = esc_2_m5_fault5.astype("float64")
esc_2_m5_fault5 = esc_2_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_fault5 = esc_2_m5_fault5[((esc_2_m5_fault5['TimeUS'] >= min) & (esc_2_m5_fault5['TimeUS'] <= max))]
esc_2_m5_fault5=esc_2_m5_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m5_fault5 = esc_3_m5_fault5.astype("float64")
esc_3_m5_fault5 = esc_3_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_fault5 = esc_3_m5_fault5[((esc_3_m5_fault5['TimeUS'] >= min) & (esc_3_m5_fault5['TimeUS'] <= max))]
esc_3_m5_fault5=esc_3_m5_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m5_fault5 = esc_4_m5_fault5.astype("float64")
esc_4_m5_fault5 = esc_4_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_fault5 = esc_4_m5_fault5[((esc_4_m5_fault5['TimeUS'] >= min) & (esc_4_m5_fault5['TimeUS'] <= max))]
esc_4_m5_fault5=esc_4_m5_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m5_fault5 = esc_5_m5_fault5.astype("float64")
esc_5_m5_fault5 = esc_5_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_fault5 = esc_5_m5_fault5[((esc_5_m5_fault5['TimeUS'] >= min) & (esc_5_m5_fault5['TimeUS'] <= max))]
esc_5_m5_fault5=esc_5_m5_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m5_fault5 = pd.read_csv("ESC_0.csv")\nesc_0_m5_fault5 = esc_0_m5_fault5.astype("float64")\nesc_0_m5_fault5 = esc_0_m5_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m5_fault5 = esc_0_m5_fault5[((esc_0_m5_fault5[\'TimeUS\'] >= min) & (esc_0_m5_fault5[\'TimeUS\'] <= max))]\nesc_0_m5_fault5=esc_0_m5_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m5_fault5 = pd.read_csv("ESC_1.csv")\nesc_1_m5_fault5 = esc_1_m5_fault5.astype("float64")\nesc_1_m5_fault5 = esc_1_m5_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m5_fault5 = esc_1_m5_fault5[((esc_1_m5_fault5[\'TimeUS\'] >= min) & (esc_1_m5_fault5[\'TimeUS\'] <= max))]\nesc_1_m5_fault5=esc_1_m5_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\nesc_2_m5_fault5 = pd.r

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m5_fault5 = imu_0_m5_fault5.astype("float64")
imu_0_m5_fault5 = imu_0_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_fault5 = imu_0_m5_fault5[((imu_0_m5_fault5['TimeUS'] >= min) & (imu_0_m5_fault5['TimeUS'] <= max))]

imu_1_m5_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m5_fault5 = imu_1_m5_fault5.astype("float64")
imu_1_m5_fault5 = imu_1_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_fault5 = imu_1_m5_fault5[((imu_1_m5_fault5['TimeUS'] >= min) & (imu_1_m5_fault5['TimeUS'] <= max))]

imu_2_m5_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m5_fault5 = imu_2_m5_fault5.astype("float64")
imu_2_m5_fault5 = imu_2_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_fault5 = imu_2_m5_fault5[((imu_2_m5_fault5['TimeUS'] >= min) & (imu_2_m5_fault5['TimeUS'] <= max))]

imu_m5_fault5 = pd.concat((imu_0_m5_fault5, imu_1_m5_fault5, imu_2_m5_fault5))
imu_m5_fault5=imu_m5_fault5.groupby(imu_m5_fault5.TimeUS, as_index=False).mean()

display(imu_m5_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76591750.0,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199
1,76594395.0,0.076037,-0.019434,-0.004237,-0.040051,-0.149500,-9.643477
2,76597003.0,-0.153429,-0.056478,-0.003201,0.009088,-0.251848,-9.482471
3,76599933.0,-0.206979,-0.032131,0.004984,-0.091977,-0.123594,-9.647797
4,76603109.0,0.003761,0.014892,0.016421,-0.148840,0.052601,-10.130823
...,...,...,...,...,...,...,...
18452,129996960.0,0.344133,0.040225,-0.040295,-0.996638,0.970077,-13.997997
18453,129999994.0,0.342809,0.087223,-0.039791,-1.069123,1.056487,-14.131667
18454,130002550.0,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180
18455,130008951.0,0.368710,0.092210,-0.050912,-1.847238,0.714070,-15.517767


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_fault5 = pd.merge_ordered(imu_m5_fault5,att_m5_fault5)
m5_fault5=m5_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m5_fault5 = pd.merge_ordered(m5_fault5, esc_0_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_1_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_2_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_3_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_4_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_5_m5_fault5)
m5_fault5=m5_fault5.fillna(method="ffill").fillna(method="bfill")'''
m5_fault5 = pd.merge_ordered(m5_fault5, rcou_m5_fault5)
m5_fault5=m5_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
m5_fault5["TimeUS"] = m5_fault5["TimeUS"] - m5_fault5.iloc[0]["TimeUS"]
m5_fault5["TimeUS"] = m5_fault5["TimeUS"].astype(int)
m5_fault5["Guasto"] = guasto

In [13]:
last_cell = m5_fault5.iloc[-1, m5_fault5.columns.get_loc('TimeUS')]
print(last_cell)

53425589


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
18695  53411615
18696  53414472
18697  53417329
18698  53420186
18699  53423043

[18700 rows x 1 columns]


In [16]:
m5_fault5_final = pd.merge_ordered(m5_fault5,df)
m5_fault5_final=m5_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m5_fault5_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.173409  0.043199  0.004685 -0.176615  0.117010 -10.015199   
1          1594  0.173409  0.043199  0.004685 -0.176615  0.117010 -10.015199   
2          2857  0.173409  0.043199  0.004685 -0.176615  0.117010 -10.015199   
3          4239  0.076037 -0.019434 -0.004237 -0.040051 -0.149500  -9.643477   
4          5714  0.076037 -0.019434 -0.004237 -0.040051 -0.149500  -9.643477   
...         ...       ...       ...       ...       ...       ...        ...   
46514  53420186  0.368710  0.092210 -0.050912 -1.847238  0.714070 -15.517767   
46515  53423043  0.368710  0.092210 -0.050912 -1.847238  0.714070 -15.517767   
46516  53424615  0.231860  0.084674 -0.042090 -2.193608  0.006728 -16.338220   
46517  53424776  0.231860  0.084674 -0.042090 -2.193608  0.006728 -16.338220   
46518  53425589  0.231860  0.084674 -0.042090 -2.193608  0.006728 -16.338220   

       DesRoll  Roll  DesPitch  Pitch  

In [17]:
m5_fault5_final = m5_fault5_final[m5_fault5_final.TimeUS.isin(to_be)]

In [18]:
m5_fault5_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
2,2857,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
4,5714,0.076037,-0.019434,-0.004237,-0.040051,-0.149500,-9.643477,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
6,8571,-0.153429,-0.056478,-0.003201,0.009088,-0.251848,-9.482471,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
9,11428,-0.206979,-0.032131,0.004984,-0.091977,-0.123594,-9.647797,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46507,53411615,0.342809,0.087223,-0.039791,-1.069123,1.056487,-14.131667,0.56,-0.34,-0.28,-2.45,5.87,1.13,1538.0,1435.0,1457.0,1520.0,1486.0,1492.0,1.0
46511,53414472,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180,0.54,-0.07,-0.25,-2.44,5.87,1.10,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1.0
46512,53417329,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180,0.54,-0.07,-0.25,-2.44,5.87,1.10,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1.0
46514,53420186,0.368710,0.092210,-0.050912,-1.847238,0.714070,-15.517767,0.54,-0.07,-0.25,-2.44,5.87,1.10,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1.0


In [19]:
from datetime import timedelta
m5_fault5_final=m5_fault5_final.reset_index(drop=True)
m5_fault5_final['TimeUS'] = pd.to_datetime(m5_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
1,00:00:00.002857,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
2,00:00:00.005714,0.076037,-0.019434,-0.004237,-0.040051,-0.149500,-9.643477,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
3,00:00:00.008571,-0.153429,-0.056478,-0.003201,0.009088,-0.251848,-9.482471,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
4,00:00:00.011428,-0.206979,-0.032131,0.004984,-0.091977,-0.123594,-9.647797,0.07,0.62,-0.03,-0.26,17.98,17.98,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18695,00:00:53.411615,0.342809,0.087223,-0.039791,-1.069123,1.056487,-14.131667,0.56,-0.34,-0.28,-2.45,5.87,1.13,1538.0,1435.0,1457.0,1520.0,1486.0,1492.0,1.0
18696,00:00:53.414472,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180,0.54,-0.07,-0.25,-2.44,5.87,1.10,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1.0
18697,00:00:53.417329,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180,0.54,-0.07,-0.25,-2.44,5.87,1.10,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1.0
18698,00:00:53.420186,0.368710,0.092210,-0.050912,-1.847238,0.714070,-15.517767,0.54,-0.07,-0.25,-2.44,5.87,1.10,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m5_fault5_final)/350)):
        V1=m5_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_25392\1053969849.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_25392\1053969849.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,0.003529,0.011127,3.299798,0.105393,77.0,58.0,6.897303,4.682781,0.064848,0.011800,-0.761803,0.126378,89.0,66.0,2.479259,1.795350,0.092842,0.004477,-1.389571,0.114384,74.0,73.0,1.777062,0.560177,0.069364,0.068834,-0.241095,0.271015,109.0,108.0,13.936845,7.944906,0.172177,0.051484,0.197449,0.284573,88.0,87.0,10.995167,9.389711,-11.128620,0.905162,-1.511269,11.169099,0.0,110.0,91.976453,36.753575,0.674644,0.023135,0.029168,0.691530,74.0,72.0,14.896988,3.454577,0.115641,1.287585,0.641017,1.138986,0.0,92.0,184.439761,15.260479,19.894957,3.629556,-1.368537,19.985708,102.0,75.0,0.012479,-1.557976,1590.270655,10189.643679,-0.249435,1593.462071,75.0,77.0,1096.408735,825.705354,1527.977208,4621.359479,0.112181,1529.484404,80.0,83.0,1961.466635,1088.968173,1587.421652,9353.678844,-0.209778,1590.356731,71.0,72.0,1481.600103,744.687256,1531.943020,4318.042458,1.297027,1533.347696,78.0,77.0,913.465716,665.515979,1558.535613,6206.723728,0.111370,1560.519880,98.0,99.0,2005.925722,1028.628144,1561.592593,7903.196402,-0.290801,1564.113840,95.0,94.0,1847.639609,853.040325,1
1,1.0,0.003542,0.011095,3.317703,0.105244,71.0,53.0,6.720523,5.221725,0.065261,0.011826,-0.769151,0.126696,95.0,70.0,2.477847,1.640515,0.092779,0.004465,-1.386821,0.114284,76.0,75.0,1.802701,0.512901,0.070988,0.069567,-0.245607,0.272779,106.0,100.0,13.747436,7.937568,0.172236,0.051338,0.206225,0.284355,96.0,95.0,11.914837,7.523635,-11.129243,0.902720,-1.506339,11.169611,0.0,120.0,93.269518,37.117652,0.674716,0.023071,0.037091,0.691554,76.0,74.0,15.697397,3.305000,0.124886,1.314005,0.630591,1.151463,0.0,92.0,186.179940,15.536004,19.904631,3.652154,-1.373822,19.995902,97.0,72.0,0.061305,-1.702421,1590.593750,10197.358707,-0.244145,1593.786958,70.0,72.0,1060.940318,894.612781,1527.869318,4612.290566,0.109106,1529.373676,75.0,78.0,1993.463257,1103.746939,1587.684659,9351.378909,-0.202318,1590.618556,69.0,70.0,1484.033985,785.922330,1531.911932,4306.080541,1.305561,1533.312758,80.0,79.0,878.409494,625.467728,1558.616477,6191.342520,0.123125,1560.595744,97.0,98.0,2028.867591,1110.112293,1561.767045,7891.392871,-0.281834,1564.284270,96.0,95.0,1836.286992,801.234832,1
2,2.0,0.003713,0.011074,3.325702,0.105151,79.0,60.0,6.590156,5.526607,0.065527,0.011818,-0.770623,0.126799,89.0,67.0,2.425313,1.321189,0.092708,0.004455,-1.384449,0.114179,70.0,69.0,1.826156,0.496696,0.072954,0.070734,-0.213868,0.275419,99.0,95.0,13.629511,8.352536,0.172146,0.051195,0.215275,0.284050,86.0,82.0,12.590929,7.593051,-11.129972,0.900343,-1.501386,11.170231,0.0,120.0,94.517706,37.402843,0.674788,0.023007,0.045016,0.691578,70.0,68.0,16.473507,3.266720,0.134079,1.340104,0.616053,1.163737,0.0,100.0,187.889180,15.765937,19.914249,3.674437,-1.379194,20.006033,106.0,77.0,0.092376,-1.00

In [23]:
path_file = path_file.replace(r"M5\FAULT_M5_5", "")
os.chdir(path_file)
df_merged.to_csv('m5_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
